In [1]:
# https://blog.naver.com/jeonghj66/222005972393 의 코드를 가져옴

import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

#하이퍼파라미터 정의
EPOCHS = 10

#네트워크 구조 정의
def MyModel():
    return Sequential([Conv2D(32, (3, 3), padding='same', activation='relu'), # 28x28x32, same=zero-padding | valid=no-padding
                       MaxPool2D(), # 14x14x32, 디폴트가 2 stripe로 크기를 절반으로 줄여준다
                       Conv2D(64, (3, 3), padding='same', activation='relu'), # 14x14x64
                       MaxPool2D(), # 7x7x64
                       Conv2D(128, (3, 3), padding='same', activation='relu'), # 7x7x128
                       Flatten(), # 7x7x128 = 6272
                       Dense(128, activation='relu'), # 6272 -> 128
                       Dense(10, activation='softmax')]) # 128 -> 10

#데이터 불러오기
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

# NHWC : 배치 + Height + Width + Channel
x_train = x_train[..., np.newaxis] # 위의 4번째 채널을 추가
x_test = x_test[..., np.newaxis] # 위의 4번째 채널을 추가

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

#모델 생성
model = MyModel()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#모델 학습
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)


Epoch 1/10
1875/1875 [==============================] - 51s 27ms/step - loss: 0.3919 - accuracy: 0.8580 - val_loss: 0.2850 - val_accuracy: 0.8960
Epoch 2/10
1875/1875 [==============================] - 61s 33ms/step - loss: 0.2500 - accuracy: 0.9076 - val_loss: 0.2532 - val_accuracy: 0.9074
Epoch 3/10
1875/1875 [==============================] - 66s 35ms/step - loss: 0.2084 - accuracy: 0.9228 - val_loss: 0.2664 - val_accuracy: 0.9044
Epoch 4/10
1875/1875 [==============================] - 59s 32ms/step - loss: 0.1746 - accuracy: 0.9344 - val_loss: 0.2511 - val_accuracy: 0.9105
Epoch 5/10
1875/1875 [==============================] - 60s 32ms/step - loss: 0.1451 - accuracy: 0.9463 - val_loss: 0.2401 - val_accuracy: 0.9146
Epoch 6/10
1875/1875 [==============================] - 60s 32ms/step - loss: 0.1187 - accuracy: 0.9560 - val_loss: 0.2376 - val_accuracy: 0.9214
Epoch 7/10
1875/1875 [==============================] - 61s 32ms/step - loss: 0.0976 - accuracy: 0.9638 - val_loss: 0.2718 -

In [ ]:
# cs231n lecture8 deep learning software 참고

import numpy as np
np.random.seed(0)
# tf v1의 문법사용을 위해
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

N, D, H = 64, 1000, 100
# placeholder - 변수를 선언 but 초기값 설정은 나중에 하겠다.
x = tf.placeholder(tf.float32, shape=(N,D))
y = tf.placeholder(tf.float32, shape=(N,D))
w1 = tf.placeholder(tf.random_normal, shape=(D,H))
w2 = tf.placeholder(tf.random_normal, shape=(H,D))

# dot product 후 Relu 활성화 함수처리 결과
h = tf.maximum(tf.matmul(x, w1), 0)
y_pred = tf.matmul(h, w2)
diff = y_pred - y
loss = tf.reduce_mean(tf.reduce_sum(diff**2, axis=1))

grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])

# 모델링 설계후 tf.Sesson()으로 확정 필요
with tf.Session() as sess:
    values = {x: np.random.randn(N, D),
             w1: np.random.randn(D, H),
             w2: np.random.randn(H, D),
             y: np.random.randn(N, D),}
    
    # training
    learning_rate = 1e-5
    for t in range(50):
        # 오차 역전파
        out = sess.run([loss, grad_w1, grad_w2], feed_dict=values)
        loss_val, grad_w1_val, grad_w2_val = out
        values[w1] -= learning_rate * grad_w1_val
        values[w2] -= learning_rate * grad_w2_val

        print(loss_val)

In [ ]:
# cs231n lecture8 deep learning software 참고

import numpy as np
np.random.seed(0)
# tf v1의 문법사용을 위해
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

N, D, H = 64, 1000, 100
# placeholder - 변수를 선언 but 초기값 설정은 나중에 하겠다.
x = tf.placeholder(tf.float32, shape=(N,D))
y = tf.placeholder(tf.float32, shape=(N,D))
w1 = tf.Variable(tf.random_normal((D,H))) # 정규분포
w2 = tf.Variable(tf.random_normal((H,D)))

# dot product 후 Relu 활성화 함수처리 결과
h = tf.maximum(tf.matmul(x, w1), 0)
y_pred = tf.matmul(h, w2)
diff = y_pred - y
loss = tf.reduce_mean(tf.reduce_sum(diff**2, axis=1))
grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])

learning_rate = 1e-5
new_w1 = w1.assign(w1 - learning_rate * grad_w1)
new_w2 = w2.assign(w2 - learning_rate * grad_w2)
update = tf.group(new_w1, new_w2) # 이문장이 빠지면 학습이 이루어 지지 않음

# 모델링 설계후 tf.Sesson()으로 확정 필요
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    values = {x: np.random.randn(N, D),
             y: np.random.randn(N, D),}
    losses = []
    
    # training
    for t in range(50):
        loss_val,_ = sess.run([loss, update], feed_dict = values)

        print(loss_val)

In [15]:
# cs231n lecture8 deep learning software 참고

import numpy as np
np.random.seed(0)
# tf v1의 문법사용을 위해
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

N, D, H = 64, 1000, 100
init = tf.contrib.layers.xavier_initializer() # tf v2가 나오면서 삭제된 함수
h = tf.layers.dense(inputs = x, units =H, activation=tf.nn.relu, kernel_initializer = init)
y_pred = tf.layers.dense(intputx = h, units=D, kerner_initializer=init)

loss = tf.losses.mean_squared_error(y_pred, y)

optimizer = tf.train.GradientDescentOptimizer(1e0)
updates = optimizer.minimize(loss)

# 모델링 설계후 tf.Sesson()으로 확정 필요
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    values = {x: np.random.randn(N, D),
             y: np.random.randn(N, D),}
    losses = []
    
    # training
    for t in range(50):
        loss_val,_ = sess.run([loss, update], feed_dict = values)

        print(loss_val)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


TypeError: dense() got an unexpected keyword argument 'intputx'

In [17]:
# cs231n lecture 8 강의 참고
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD

N, D, H = 64, 1000, 100
model = Sequential()
model.add(Dense(input_dim = D, output_dim = H)) # 완전 연결
model.add(Activation('relu'))                   # 활성화 함수
model.add(Dense(input_dim=H, output_dim = D))

optimizer = SGD(lr = 1e0)
model.compile(loss = 'mean_squared_error', optimizer = optimizer)

x = np.random.randn(N, D)
y = np.random.randn(N, D)
history = model.fit(x, y, nb_epoch=50, batch_size=N, verbose=0)

TypeError: __init__() missing 1 required positional argument: 'units'